# Evaluation Entry

**Note**: This notebook generate results for evaluations. Evaluation results please go to folder 'eval_train_size'

### default nerual network parameters
~~~~
self.nn_params = {
    'hidden_size': 100,
    'depth': 3,               # network depth = 'depth' + 1
    'activation': torch.nn.Tanh,
    'device': device
}
~~~~
### default training parameters
~~~~
self.train_params = {
    'epochs': 500,
    'train_loss': [
        'mse_loss', 
        'phy_loss', 
        'energy_loss'
    ],
    'test_loss': [
        'phy_loss', 
        'energy_loss'
    ],
    'num_batch': 1,
    'optimizer': torch.optim.Adamax,
    'cyclical': {
        'base_lr': 0.001, 
        'max_lr': 0.1, 
        'step_size_up': 20, 
        'step_size_down': 20, 
        'mode': 'triangular'
    },     # set to False or {} to disable
    'L2_reg': 0.0,
    'verbose': False,
    'print_interval': 10,
    'early_stopping': False
}
~~~~
### default input/output parameters
~~~~
self.io_params = {
    'path_out': '../results/',
    'path_fig': '../figures/',
    'path_log': '../logs/',
    'env_path': [],           # path that will be registered through sys.path
    'use_timestamp': True
}
~~~~
### default dataset parameters
~~~~
self.data_params = {
    'data_path': '../datasets/Electromagnetic/dataShort',
    'normalize_input': True,
    'normalize_output': False,
    'device': device,
    'dataset': 'new'
}
~~~~
### default loss function parameters
~~~~
self.loss_params = {
    'lambda_s': 1.0,
    'lambda_e0': 0.2,
    'anneal_factor': 0.9,
    'anneal_interval': 50,    # this is a parameter also for noise and cyclical update 
    'norm_wf': False,
    # set to False or None or {} to disable the noise
    'noise': {'mode': 'uniform', 'mean': 0.0, 'var': 0.5, 'decay': 0.9}, 
    'cyclical': {'mode': 'sin', 'decay': 0.9, 'mean': 1.0, 'amp': 1.0, 'period': 20, 'cycle_momentum': False}
}
~~~~

## The Models

|           Models            |      Abbr.       |
| --------------------------- | ---------------- |
| Deep Neural Networks        | DNN              |
| Normalized SE-DNN$_{ex}$    | NSE-DNN$_{ex}$   |

In [ ]:
%matplotlib inline

import seaborn as sns
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('../scripts/')

from training_electromagnetic import Trainer
from presets import LambdaSearch
from config_plots import global_settings
from utils import *
from parameters import Params

global_settings()

from fastprogress.fastprogress import master_bar, progress_bar

torch.cuda.set_device(6)
device = torch.device('cuda:6')

## Free Log Files

In [ ]:
num_logs = len(
    [
        name for name in os.listdir(
            '../logs/'
        ) if os.path.isfile(name)
    ]
)

if num_logs > 2000:
    free_logs('../logs/')

## Set Up Parameters and Start Tasks

This part includes customized training parameters. Change these to whatever you need.

In [ ]:
param_presets = LambdaSearch(data_path='//home/elhamod/CoPhy-PGNN/datasets')

train_sizes = ['2000examples_400x400complex.mat'] #Other datasets ['dataShort.mat', 'data1000.mat', 'dataFull.mat']

datasets = []
break_loop = True
loss_plot = False


num_of_experiments = 1

mb = master_bar(range(num_of_experiments))
for i in mb:
    for train_size in progress_bar(train_sizes):
        
        param = param_presets.NSE_DNNex()
        param.data_params['train_size']=train_size
        param.name = 'CoPhy'
        param.train_params['break_loop_early'] = break_loop
        param.train_params['num_batch'] = 10
        param.loss_params['lambda_e0'] = 0.01# Initial value of S_coeff
        param.loss_params['anneal_factor'] = 0.9# A lower value will decay S_coeff faster.
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.00000006,# Final value of C_coeff
            'threshold': 1500,
            'smooth': 0.003
        }   
        param.train_params['epochs'] = 4000
        param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
        param.nn_params['hidden_size'] = 100
        param.nn_params['depth'] = 2
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

param_presets = LambdaSearch(data_path='../datasets/')

train_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, 15000, 20000]
break_loop = False
loss_plot = False
mb = master_bar(range(10))
for i in mb:
    for train_size in progress_bar(train_sizes):
        
        # the black-box neural networks
        param = param_presets.DNN()
        param.data_params['train_size']=train_size
        param.name = 'BB'
        param.train_params['break_loop_early'] = break_loop
        param.train_params['num_batch'] = 10
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.train_params['epochs'] = 4000
        param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
        param.nn_params['hidden_size'] = 100
        param.nn_params['depth'] = 2
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)
        
        # CoPhy-PGNN
        param = param_presets.NSE_DNNex()
        param.data_params['train_size']=train_size
        param.name = 'CoPhy'
        param.train_params['break_loop_early'] = break_loop
        param.train_params['num_batch'] = 10
        param.loss_params['lambda_e0'] = 0.01# Initial value of S_coeff
        param.loss_params['anneal_factor'] = 0.9# A lower value will decay S_coeff faster.
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.00000006,# Final value of C_coeff
            'threshold': 1500,
            'smooth': 0.003
        }   
        param.train_params['epochs'] = 4000
        param.train_params['early_stopping'] = {'patience':4000, 'verbose':False, 'delta':0}
        param.nn_params['hidden_size'] = 100
        param.nn_params['depth'] = 2
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)